# 1. Install and Import Dependencies

In [1]:
!pip3 install torch torchvision torchaudio


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [2]:
!pip install 'stable-baselines3[extra]' 'protobuf==3.20.*'


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [3]:
!pip install gym mss pyautogui pytesseract


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [4]:
!pip install gymnasium


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [5]:
#chrome://dino/

In [6]:
from mss import mss
import pyautogui
import cv2
import numpy as np
import pytesseract
from matplotlib import pyplot as plt
from time import sleep
import gymnasium as gym
from gymnasium import Env
from gymnasium.spaces import Box, Discrete 
import re

# 2. Build the Environment

## 2.1 Create Environment

In [7]:
class WebGame(Env):
    ## THIS SCREEN HOLDS ALL THE INFORMATION 
    ## TO UNDERSTAND THE STATE OF THE GAME:
    ## TIME_ELAPSED, REMAINING_BALLS_COUNT, SCORE ETC.
    entire_screen = {'width': 670, 'height': 584}
    
    #### A CONCENTRATED AREA WHERE THERE ARE NO DISTRACTIONS ####
    ###### LIKE SCORE, TIME_ELAPSED, REMAINING_BALLS_COUNT.######
    ################## IT'S A PURE PLAYGROUND ###################
    game_play_screen = {'height':584 ,'width':380}
    
    ## THIS IS THE LOCATION I USE TO UNDERSTAND IF THE GAME IS OVER OR NOT (I CHOSED IT TO BE "NEW GAME" button)
    gameover_location = {}
    ## THIS IS THE LOCATION I USE TO UNDERSTAND ELAPSED TIME IN THE GAME REPRESENTED IN SECONDS.
    time_location = {}
    
    ##### COORDINATES OF THE VIDEO GAME #####
    ####### W/ RESPECT TO YOUR SCREEN #######
    # 353: HORIZONTAL MIDDLE POSITION OF GAME SCREEN
    # 690: RIGHT ABOVE THE BALL YOU ARE SHOOTING WITH TO COVER A GOOD SHOOTING ANGLE
    shooting_coordinates = {'x':353 ,'y': 690}
    start_new_game_coordinates = {'x':353, 'y':540}

    # SIZE OF THE STEP TAKEN FOR 'move_left' AND 'move_right' ACTIONS
    steps = 5
    
    ####### I USE THESE BORDERS AS A WALL SO THAT #######
    ## CURSOR TO CAN NOT GO BEYOND THE GAME PLAYGROUND ##
    wall_borders = {'left':180, 'right':520}
    
    def __init__(self):
        super().__init__()
        ## Setup spaces
        # self.observation_space = Box(low=0, high=255, shape=(1,83,100), dtype=np.uint8)
        ## Define the observation space (example placeholder)
        self.observation_space = Box(low=0, high=255, shape=(1, self.game_play_screen['height'], self.game_play_screen['width']), dtype=np.uint8)
        
        # move_left, move_right, shoot, swap ball, no-op
        self.action_space = Discrete(5)

        # Capture game frames
        self.cap = mss()
        self.swap_button_location = {'x':475, 'y':730} # DO NOT SWITCH SWAP BUTTON TO RELATIVE POSITIONING
        
        #################### NOTE ###########################
        ### I THINK LOCATIONS SHOULD BE MOVED TO BE CLASS ###
        ### VARIABLES BECAUSE I WOULD WANNA CHANGE THESE ####
        ### VARIABLES XROSS ALL INSTANCES OF THIS CLASS #####
        #############################################################################################
        ############ Entire_screen includes: Score, game-area, timer, ############
        ### remaining_ball_count, swap_button. The rest of the sections ##########
        ### will be extracted from the Entire_screen with relative positions, ####
        ################### to limit ss actions to 1 per step. ###################
        
        self.entire_screen_location = {'top': 170, 'left': 20, 'width': self.entire_screen['width'], 'height': self.entire_screen['height']}    
        #############################################################################################
        ########### THESE HAVE CHANGED TO BE RELATIVE TO entire_screen_location ####################################################
        # self.game_location = {'top': 170, 'left': 190, 'width': self.game_play_screen['width'], 'height': self.screen_heightgame_play_screen['height']}
        # self.gameover_location = {'top': 520, 'left': 260, 'width': 180, 'height': 40}
        # self.score_location = {'top': 232, 'left': 20, 'width': 118, 'height': 20}
        # self.time_location = {'top': 200, 'left': 620, 'width': 70, 'height': 60}
        # self.remaining_balls_location = {'top':710 ,'left':125, 'width':120 , 'height': 40}
        self.game_location = {'top':50 ,'left':290, 'width':800, 'height': 1200}
        self.gameover_location = {'top':690 ,'left':480, 'width':350, 'height':100}
        self.score_location = {'top':120 ,'left':10, 'width':220 , 'height': 50}
        self.time_location = {'top':85 ,'left':1200, 'width':260 , 'height': 70}
        self.remaining_balls_location = {'top':1050 ,'left':205, 'width':260 , 'height': 1000}

        #############################################################################################
        #############################################################################################

        #########################################
        ###### VARIABLES NEEDING reset() ########
        #########################################
        self.is_board_cleared = False
        self.is_terminated = False
        self.is_truncated = False
        self.previous_score = 0
        self.elapsed_seconds = 0
        self.remaining_balls_count = 100
        self.score = 0
        self.entire_screen_raw = [] ## look at here maybe this should be self.get_entire_screen instead of []
        ###############################
        ###############################
    
    # Function to move the mouse left
    def move_left(self):
        current_x, current_y = pyautogui.position()
        new_x = max(self.wall_borders['left'], current_x - self.steps)
        pyautogui.moveTo(new_x, self.shooting_coordinates['y'])
    
    # Function to move the mouse right
    def move_right(self):
        current_x, current_y = pyautogui.position()
        new_x = min(self.wall_borders['right'], current_x + self.steps)
        pyautogui.moveTo(new_x, self.shooting_coordinates['y'])
    
    # Function to perform a click (shoot)
    def shoot(self):
        x, y = pyautogui.position()
        pyautogui.click(x=x, y=y)

    # Function to swap the shooting ball
    def swap_ball(self):
        x_before_jump, y_before_jump = pyautogui.position()
        print("moving to location to swap: ",self.swap_button_location)
        pyautogui.click(x=self.swap_button_location["x"], y=self.swap_button_location["y"], button='left')
        print("swap done moving back to where i was just the moment ago")
        pyautogui.moveTo(x=x_before_jump, y=y_before_jump)
    
    # Function to do nothing
    def no_op(self):
        pass  # Perform no action

    def calculate_reward(self):
        
        # Basic reward: change in score
        score_reward = self.score - self.previous_score
        approximate_number_of_bubbles_popped = score_reward // 100;
        # Additional reward for popping more bubbles in one shot
        # For instance, if more than 3 bubbles are popped, apply a multiplier
        if approximate_number_of_bubbles_popped > 3:
            efficiency_bonus = 1.25
        else:
            efficiency_bonus = 1.0
        # Add to the total_reward
        total_reward = score_reward * efficiency_bonus
        self.previous_score = self.score

        # the more time elapses the more penalty there is.
        # (because each time it's called the time_elapse will be a higher value)
        # chose this penalty formula to prioritize clearing the board faster.
        # changable it if it doesn't work good. Maybe a fixed value will do fine?
        time_penalty_constant = 100
        total_reward = total_reward - self.get_elapsed_seconds() * time_penalty_constant

        # cleared board bonus:
        # is_board_cleared = self.update_is_board_cleared()
        if self.is_board_cleared:
            total_reward += 1000;
            
        # remaining balls bonus: more remains more bonus
        remaining_balls_reward_constant = 100
        if self.is_terminated and self.remaining_balls_count > 0:
            total_reward += self.remaining_balls_count * 100
        return total_reward
        
    def get_remaining_balls_count(self):
        section = self.entire_screen_raw[self.remaining_balls_location['top']:self.remaining_balls_location['top']+self.remaining_balls_location['height'], self.remaining_balls_location['left']:self.remaining_balls_location['left']+self.remaining_balls_location['width']]
        text = pytesseract.image_to_string(section)[:4]
        potential_number_string = self.extract_numbers(text)
        if len(potential_number_string):
            self.remaining_balls_count = int(potential_number_string)
        return self.remaining_balls_count
        
    def step(self, action):
        ## Map the action to the corresponding function
        action_map = {
            0: self.move_left,
            1: self.move_right,
            2: self.shoot,
            3: self.swap_ball,
            4: self.no_op,
        }
        
        #### Perform the action ####
        ############################
        if action in action_map:
            action_map[action]() # Call the corresponding function
        ############################
        
        observation, raw, gray = self.get_observation()
        terminated = self.update_is_terminated()

        #### Reward is calculated after observation ####
        ################################################
        reward = self.calculate_reward()
        info = {}
        truncated = False # not decided how to use this yet.
        return observation, reward, terminated, truncated, info
        ################################################
        
    def reset(self, seed=None, options=None):
        # super().reset(seed=seed)
        # Clicking on allegedly "New Game" on the screen
        pyautogui.click(x=self.start_new_game_coordinates['x'], y=self.start_new_game_coordinates['y'], button='left', clicks=1)
        # Positioning cursor to the middle before game starts
        pyautogui.moveTo(x=self.shooting_coordinates['x'], y=self.shooting_coordinates['y']) # move to the center to start the game at the lower center
        info = {}
        # Should reset the variables that changes while playing the round.
        self.remaining_balls_count = 1000 # just a large number
        self.previous_score = 0
        self.elapsed_seconds = 0
        self.is_board_cleared = False
        self.is_terminated = False
        self.is_truncated = False
        self.score = 0
        observation, *_ = self.get_observation()
        return observation, info
        
    def render(self, area):
        raw = np.array(self.cap.grab(area))[:,:,:3].astype(np.uint8)
        plt.imshow(raw)
         
    def close(self):
        cv2.destroyAllWindows()

    def get_entire_screen(self):
        self.entire_screen_raw = np.array(self.cap.grab(self.entire_screen_location))[:,:,:3].astype(np.uint8)
        return self.entire_screen_raw
    
    def get_observation(self):
        self.entire_screen_raw = self.get_entire_screen()
        ################ ALL GAME_SCREEN RELATED ################
        game_play_raw = self.entire_screen_raw[self.game_location['top']:self.game_location['top']+self.game_location['height'], self.game_location['left']:self.game_location['left']+self.game_location['width']]
        game_play_gray = cv2.cvtColor(game_play_raw, cv2.COLOR_BGR2GRAY)
        game_play_resized = cv2.resize(game_play_gray, (self.game_play_screen['width'], self.game_play_screen['height']))
        game_play_channel = np.reshape(game_play_resized, (1, self.game_play_screen['height'], self.game_play_screen['width']))
        observation = game_play_channel
        
        ################### UPDATES INSTANCE VARIABLES ####################
        ###### NOTE: ORDER MATTERS BECAUSE ONE DEPENDENT ON ANOTHER #######
        self.remaining_balls_count = self.get_remaining_balls_count()
        self.is_terminated = self.update_is_terminated()
        self.is_board_cleared = self.update_is_board_cleared()
        ###################################################################
        
        return observation, game_play_raw, game_play_gray

    def update_is_board_cleared(self):
        if self.is_terminated and self.remaining_balls_count > 0:
            self.is_board_cleared = True
        return self.is_board_cleared
        
    def update_is_terminated(self):
        terminated_strings = ['Play', 'Now', 'Play Now', 'New', 'Game', 'New Game']
        section = self.entire_screen_raw[self.gameover_location['top']:self.gameover_location['top']+self.gameover_location['height'], self.gameover_location['left']:self.gameover_location['left']+self.gameover_location['width']]
        text = pytesseract.image_to_string(section)[:4].strip()
        if text in terminated_strings:
            self.is_terminated = True
            print('game over')
        return self.is_terminated

    def extract_numbers(self, input_string):
        print("input_string: ",input_string)
        # This regex pattern matches any character that is NOT a digit
        pattern = r'\D'
    
        # Replace all non-digit characters with an empty string
        print("extracted numbers: ",re.sub(pattern, '', input_string))
        return re.sub(pattern, '', input_string)

    def update_score(self):
        section = self.entire_screen_raw[self.score_location['top']:self.score_location['top']+self.score_location['height'], self.score_location['left']:self.score_location['left']+self.score_location['width']]
        text = self.extract_numbers(pytesseract.image_to_string(section)[:4])
        if len(text) == 0:
            self.score = 0
        else:
            self.score = int(text)
        return self.score

    def extract_numbers_and_colon(self, that_string):
        # Regular expression pattern: matches digits and colons
        pattern = r'[0-9:]'
        # Find all matches in the string
        matches = re.findall(pattern, that_string)
        # Join matches into a single that_string
        return ''.join(matches)

    def convert_to_seconds(self, timer_str):
        timer_str = self.extract_numbers_and_colon(timer_str)
        timers = timer_str.split(":")
        if len(timers) == 3:
            hours, minutes, seconds = timers
            minutes = int(minutes) + 60 * int(hours)
        elif len(timers) == 2:
            minutes, seconds = timers
        total = int(minutes) * 60 + int(seconds)
        return total
    
    def get_elapsed_seconds(self):
        timer_location = {'top':85 ,'left':1200, 'width':260 , 'height': 70}
        section = self.entire_screen_raw[timer_location['top']:timer_location['top']+timer_location['height'], timer_location['left']:timer_location['left']+timer_location['width']]
        time = pytesseract.image_to_string(section)
        total_in_seconds = 0
        if not len(time):
            return self.elapsed_seconds
        else:
            self.elapsed_seconds = self.convert_to_seconds(time)
        return self.elapsed_seconds

    # This is a dummy click just so that cursor is switched to the games' browser
    def switch_to_games_browser(self):
        pyautogui.click(x=self.start_new_game_coordinates['x'], y=self.start_new_game_coordinates['y'], button='left', clicks=1)
        

# 2.2 Test Environment

In [8]:
env = WebGame()

In [9]:
#### This section is mostly used to understand if we captured 
#### the correct frame for each feature
# entire_screen_raw = env.get_entire_screen()
# is_term = env.update_is_terminated()
# print("seconds: ",seconds)

# plt.imshow(raw)
# env.get_remaining_balls_count(entire_screen_raw, cv2.cvtColor(entire_screen_raw, cv2.COLOR_BGR2GRAY))
# print("env.remaining_balls_counts: ",env.remaining_balls_count)

In [10]:
#####################################################################################################
#### THIS IS FOR SANITY CHECKING THE SCREEN POSITIONS OF THE KEY METRICS LIKE SCORE, TIMER ETC.. ####
#####################################################################################################
# swap_button_location = env.swap_button_location # {'x':475, 'y':710}
# entire_screen['width'] = 670
# entire_screen['height'] = 584
# entire_screen_location = {'top': 170, 'left': 20, 'width': entire_screen['width'], 'height': entire_screen['height']}
# env.render(entire_screen_location)


# game_location = env.game_location # {'top': 170, 'left': 184, 'width': env.game_play_screen['width'], 'height': env.game_play_screen['height']}
# score_location = {'top': 232, 'left': 20, 'width': 118, 'height': 20}
# gameover_location = env.gameover_location # {'top': 520, 'left': 260, 'width': 180, 'height': 40}
# time_location = env.time_location # {'top': 200, 'left': 620, 'width': 70, 'height': 60}
# remaining_balls_location = env.remaining_balls_location # {'top':710 ,'left':125, 'width':120 , 'height': 40}

####################################
#### REMDER ONE AREA AT A TIME. ####
####################################
# # swap button
# pyautogui.click(x=swap_button_location['x'], y=swap_button_location['y'])
# # game play area
# game_location = {'top': 170, 'left': 40, 'width': 400, 'height': entire_screen['height']}
# print("game_location")
# env.render(game_location)
# # score
# print("score_location")
# env.render(score_location)
# # time elapsed
# print("time_location")
# env.render(time_location)
# # remaining balls
# print("remaining_balls_location")
# env.render(remaining_balls_location)
# # # Hit New Game
# print("gameover_location")
# env.render(gameover_location)

In [11]:
#######################################################################################################################
#### EXPERIMENTS CELL: This cell is used for observing how the cursor was moving according to these specifications ####
#######################################################################################################################
# print("starting the game")
# pyautogui.click(x=env.start_new_game_coordinates['x'], y=env.start_new_game_coordinates['y'], button='left', clicks=2, interval=0.25) # click on New Game, clicks twice because first click goes towards selecting the browser
# sleep(1)

# print("moving to shooting point")
# pyautogui.moveTo(x=env.shooting_coordinates['x'], y=env.shooting_coordinates['y'])
# print("peww from center")
# env.click()
# sleep(1)

# print("moving right")
# for i in range(0, 10):
#     sleep(1)
#     env.move_right()
# print("peww from right")
# env.click()

# print("moving left")
# for i in range(0, 20):
#     sleep(1)
#     env.move_left()
# print("peww from left")
# env.click()

# print("swapping balls")
# sleep(1)
# env.swap_ball()


In [12]:
# plt.imshow(cv2.cvtColor(obs[0], cv2.COLOR_GRAY2BGR))

In [13]:
############################################################
### This is just a dummy loop to see the game lifecycle. ###
############################################################
# env.switch_to_games_browser()
# env.reset()
# for episode in range(10):
#     obs = env.reset()
#     terminated = False
#     truncated = False
#     total_reward = 0
#     while not (terminated or truncated):
#         obs, reward, terminated, truncated, info =  env.step(env.action_space.sample())
#         print(f'reward: {reward}')
#         total_reward  += reward
#     print('Total Reward for episode {} is {}'.format(episode, total_reward))
    

# 3. Train the Model   

 ## 3.1 Create Callback 

In [14]:
# Import os for file path management
import os 
# Import Base Callback for saving models
from stable_baselines3.common.callbacks import BaseCallback
# Check Environment    
from stable_baselines3.common import env_checker

In [15]:
env_checker.check_env(env)

input_string:  
extracted numbers:  
game over
input_string:  
extracted numbers:  
game over
input_string:  
extracted numbers:  
input_string:  34

extracted numbers:  34
input_string:  34

extracted numbers:  34
input_string:  34

extracted numbers:  34
input_string:  34

extracted numbers:  34
moving to location to swap:  {'x': 475, 'y': 730}
swap done moving back to where i was just the moment ago
input_string:  34

extracted numbers:  34
input_string:  34

extracted numbers:  34
input_string:  34

extracted numbers:  34
moving to location to swap:  {'x': 475, 'y': 730}
swap done moving back to where i was just the moment ago
input_string:  34

extracted numbers:  34
moving to location to swap:  {'x': 475, 'y': 730}
swap done moving back to where i was just the moment ago
input_string:  34

extracted numbers:  34
input_string:  34

extracted numbers:  34
input_string:  33

extracted numbers:  33


In [16]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [17]:
CHECKPOINT_DIR = '/Volumes/Elements/Tuna/bubble-shooter/train/'
LOG_DIR = '/Volumes/Elements/Tuna/bubble-shooter/logs/'

In [18]:
callback = TrainAndLoggingCallback(check_freq=1000, save_path=CHECKPOINT_DIR)

## 3.2 Build DQN and Train

In [19]:
from stable_baselines3 import DQN
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack

In [20]:
env = WebGame()

In [21]:
model = DQN('CnnPolicy', env, tensorboard_log=LOG_DIR, verbose=1, buffer_size=1200000, learning_starts=1000)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/usr/local/lib/python3.11/site-packages/stable_baselines3/common/buffers.py:241: UserWarning: This system does not have apparently enough memory to store the complete replay buffer 532.63GB > 2.93GB
  warnings.warn(


In [22]:
model.learn(total_timesteps=100000, callback=callback)

input_string:  32

extracted numbers:  32
Logging to ./logs/DQN_1
input_string:  32

extracted numbers:  32
input_string:  32

extracted numbers:  32
input_string:  32

extracted numbers:  32
input_string:  31

extracted numbers:  31
moving to location to swap:  {'x': 475, 'y': 730}
swap done moving back to where i was just the moment ago
input_string:  31

extracted numbers:  31
moving to location to swap:  {'x': 475, 'y': 730}
swap done moving back to where i was just the moment ago
input_string:  31

extracted numbers:  31
input_string:  31

extracted numbers:  31
input_string:  31

extracted numbers:  31
moving to location to swap:  {'x': 475, 'y': 730}
swap done moving back to where i was just the moment ago
input_string:  31

extracted numbers:  31
input_string:  30

extracted numbers:  30
input_string:  30

extracted numbers:  30
input_string:  30

extracted numbers:  30
moving to location to swap:  {'x': 475, 'y': 730}
swap done moving back to where i was just the moment ago
in

RuntimeError: [enforce fail at inline_container.cc:424] . unexpected pos 398250944 vs 398250832

In [ ]:
model.load('train_first/best_mode l_50000') 

# 4. Test out Model

In [ ]:
for episode in range(5): 
    obs = env.reset()
    done = False
    total_reward = 0
    while not done: 
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(int(action))
        time.sleep(0.01)
        total_reward += reward
    print('Total Reward for episode {} is {}'.format(episode, total_reward))
    time.sleep(2)